In [1]:
from tkinter import messagebox
from tkinter import *
from tkinter import simpledialog
import tkinter
from tkinter import filedialog
from tkinter.filedialog import askopenfilename
import cv2
import random
import numpy as np
#from keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import to_categorical
from keras.layers import  MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D
from keras.models import Sequential
from keras.models import model_from_json
import pickle
import os
import pandas as pd
import matplotlib.pyplot as plt


main = tkinter.Tk()
main.title("Convolutional Neural Networks Model for Parkinson Disease Detection from Images")
main.geometry("1300x1200")

global filename
global image_classifier

def loadDLModel():
    global image_classifier
    with open('model/images_model.json', "r") as json_file:
        loaded_model_json = json_file.read()
        image_classifier = model_from_json(loaded_model_json)
    json_file.close()    
    image_classifier.load_weights("model/images_model_weights.h5")
    image_classifier._make_predict_function()
    print(image_classifier.summary())
    f = open('model/images_history.pckl', 'rb')
    data = pickle.load(f)
    f.close()
    acc = data['accuracy']
    accuracy = acc[48]*100
    text.insert(END,"Deep CNN accuracy ")
    text.insert(END,str(accuracy))

def imageDetection():
    global image_classifier
    labels = ['Healthy','Parkinson']
    filename = filedialog.askopenfilename(initialdir="testImages")
    image = cv2.imread(filename)
    img = cv2.resize(image, (64,64))
    im2arr = np.array(img)
    im2arr = im2arr.reshape(1,64,64,3)
    img = np.asarray(im2arr)
    img = img.astype('float32')
    img = img/255
    preds = image_classifier.predict(img)
    predict = np.argmax(preds)
    
    
    img = cv2.imread(filename)
    img = cv2.resize(img, (500,400))
    cv2.putText(img, 'Image Data Predicted as : '+labels[predict], (10, 25),  cv2.FONT_HERSHEY_SIMPLEX,0.7, (255, 0, 0), 2)
    cv2.imshow('Image Data Predicted as : '+labels[predict], img)
    cv2.waitKey(0)

def graph():
    f = open('model/images_history.pckl', 'rb')
    image = pickle.load(f)
    f.close()

    img_accuracy = image['accuracy']
    img_loss = image['loss']

    plt.figure(figsize=(10,6))
    plt.grid(True)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy/Loss')
    plt.plot(img_accuracy, 'ro-', color = 'red')
    plt.plot(img_loss, 'ro-', color = 'green')
    plt.legend(['Training Accuracy', 'Training Loss'], loc='upper left')
    plt.title('Performance Comparison')
    plt.show()

def close():
    main.destroy()
    
font = ('times', 16, 'bold')
title = Label(main, text='Convolutional Neural Networks Model for Parkinson Disease Detection from Images',anchor=W, justify=CENTER)
title.config(bg='yellow4', fg='white')  
title.config(font=font)           
title.config(height=3, width=120)       
title.place(x=0,y=5)


font1 = ('times', 13, 'bold')
loadButton = Button(main, text="Load Deep CNN model", command=loadDLModel)
loadButton.place(x=50,y=100)
loadButton.config(font=font1)  



imageButton = Button(main, text="Detect Parkinson", command=imageDetection)
imageButton.place(x=50,y=150)
imageButton.config(font=font1)  


graphButton = Button(main, text="Performance Graph", command=graph)
graphButton.place(x=50,y=200)
graphButton.config(font=font1)

exitButton = Button(main, text="Exit", command=close)
exitButton.place(x=50,y=250)
exitButton.config(font=font1)


font1 = ('times', 12, 'bold')
text=Text(main,height=20,width=78)
scroll=Scrollbar(text)
text.configure(yscrollcommand=scroll.set)
text.place(x=450,y=100)
text.config(font=font1)

main.config(bg='Sky Blue')
main.mainloop()

